# Importing Libraries

In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
import numpy as np
import pandas as pd
import imageio
import random
import matplotlib.pyplot as plt
import keras
import tensorflow.keras as K
import tensorflow.keras.backend as Kback

from keras import layers
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import Dropout
from keras.layers import Conv2DTranspose
from keras.layers import concatenate
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.callbacks import EarlyStopping
from keras.applications.resnet import ResNet50
from keras.applications import vgg16
from keras.applications import inception_v3
from keras.src.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications import DenseNet121, NASNetMobile, EfficientNetB0, Xception
from keras.layers import GlobalAveragePooling2D
from keras.applications.densenet import DenseNet121

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight


from collections import Counter
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_auc_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Loadng


In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Define the base path to your dataset
base_path = '/content/drive/My Drive/LUNGCANCER/IQ-OTH/The IQ-OTHNCCD lung cancer dataset'

# List of class names (A, B, C)
class_names =  ["Normal cases", "Malignant cases", "Bengin cases"]

# Initialize empty lists for images and labels
all_images = []
all_labels = []

# Load images and assign labels
for label, class_name in enumerate(class_names):
    class_path = os.path.join(base_path, class_name)
    image_files = os.listdir(class_path)

    for image_file in image_files:
        image_path = os.path.join(class_path, image_file)
        img = cv2.imread(image_path, cv2.IMREAD_COLOR)  # Read as GBR
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)


        # img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
        img = cv2.resize(img, (224, 224))  # Resize to desired dimensions
        img = img / 255.0  # Normalize pixel values (assuming 8-bit images)
        all_images.append(img)
        all_labels.append(label)


In [4]:
data=all_images
labels=all_labels

In [5]:
data, labels = shuffle(data, labels, random_state=42)


print('Data length:', len(data))
print('labels counts:', Counter(labels))

X = np.array(data).reshape(-1, 224, 224, 3)
y = np.array(labels)

Data length: 1097
labels counts: Counter({1: 561, 0: 416, 2: 120})


# Train Test Valid Split

In [6]:
X_train, X_mix, y_train, y_mix = train_test_split(X, y, test_size=0.2, random_state=10)
X_test, X_valid, y_test, y_valid = train_test_split(X_mix, y_mix, test_size=0.2, random_state=10)
print('Train length:', len(X_train), X_train.shape)
print('Valid length:', len(X_valid), X_valid.shape)
print('Test length:', len(X_test), X_test)
train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()
# X,y valid are used for evaluate matrix

print('Train length:', len(X_train), X_train.shape)
print('Valid length:', len(X_valid), X_valid.shape)
print('Test length:', len(X_test), X_test.shape)



# train_generator = train_datagen.flow(X_fold, y_fold, batch_size=8, shuffle=True)
# validation_generator = val_datagen.flow(X_valid, y_valid, batch_size=8, shuffle=True)

Train length: 877 (877, 224, 224, 3)
Valid length: 44 (44, 224, 224, 3)
Test length: 176 [[[[0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   ...
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]]

  [[0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   ...
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]]

  [[0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   ...
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]]

  ...

  [[0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.14901961]
   ...
   [0.14901961 0.14901961 0.14901961]
   [0.14901961 0.14901961 0.149019

#T-B/L-R Split

In [24]:
tl_train = []
tl_valid = []
tl_test=[]
for i in range(len(X_train)):
    tl_train.append(cv2.resize(X_train[i][:112, :112],(128,128)))
tl_train = np.array(tl_train)

for i in range(len(X_valid)):
  tl_valid.append(cv2.resize(X_valid[i][:112, :112],(128,128)))
tl_valid=np.array(tl_valid)

for i in range(len(X_test)):
  tl_test.append(cv2.resize(X_test[i][:112, :112],(128,128)))
tl_test=np.array(tl_test)

In [25]:
X_train.shape

(877, 224, 224, 3)

In [26]:
tl_train.shape

(877, 128, 128, 3)

In [27]:
tl_valid.shape

(44, 128, 128, 3)

In [28]:
tl_test.shape

(176, 128, 128, 3)

In [29]:
tr_train = []
for i in range(len(X_train)):

    tr_train.append(cv2.resize(X_train[i][:112, 112:],(128,128)))
tr_train = np.array(tr_train)



tr_valid=[]
for i in range(len(X_valid)):
  tr_valid.append(cv2.resize(X_valid[i][:112, 112:],(128,128)))
tr_valid=np.array(tr_valid)

tr_test=[]
for i in range(len(X_test)):
  tr_test.append(cv2.resize(X_test[i][:112, 112:],(128,128)))
tr_test=np.array(tr_test)

In [30]:
bl_train = []
for i in range(len(X_train)):
    bl_train.append(cv2.resize(X_train[i][112:, :112],(128,128)))
bl_train = np.array(bl_train)

bl_valid=[]
for i in range(len(X_valid)):
  bl_valid.append(cv2.resize(X_valid[i][112:, :112],(128,128)))
bl_valid=np.array(bl_valid)

bl_test=[]
for i in range(len(X_test)):
  bl_test.append(cv2.resize(X_test[i][112:, :112],(128,128)))
bl_test=np.array(bl_test)

In [31]:
br_train = []
for i in range(len(X_train)):
    br_train.append(cv2.resize(X_train[i][112:, 112:],(128,128)))
br_train = np.array(br_train)

br_valid = []
for i in range(len(X_valid)):
    br_valid.append(cv2.resize(X_valid[i][112:, 112:],(128,128)))
br_valid = np.array(br_valid)

br_test = []
for i in range(len(X_test)):
    br_test.append(cv2.resize(X_test[i][112:, 112:],(128,128)))
br_test = np.array(br_test)


# Features T-B/L-R + Combining Features


In [32]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(128,128,3)
)

9406464/9406464 [==============================] - 0s 0us/step


## Intermediate Model

In [33]:
NUM_CLASSES = 3
input_layer = layers.Input(shape=(128, 128, 3))
densenet_model = Sequential()
densenet_model.add(input_layer)
densenet_model.add(Conv2D(3, (3, 3), padding='same'))
densenet_model.add(base_model)
densenet_model.add(layers.Flatten(name='flatten'))
# intermediate_features = densenet_model.layers[-1].output
# attention_weights = layers.Dense(1, activation='softmax')(intermediate_features)
# attention_weights = layers.Reshape((1, 1, -1))(attention_weights)
# densenet_model.add(layers.Dense(10, activation='relu'))
densenet_model.layers[0].trainable = False

In [34]:
densenet_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 128, 128, 3)       84        
                                                                 
 mobilenetv2_1.00_128 (Func  (None, 4, 4, 1280)        2257984   
 tional)                                                         
                                                                 
 flatten (Flatten)           (None, 20480)             0         
                                                                 
Total params: 2258068 (8.61 MB)
Trainable params: 2223872 (8.48 MB)
Non-trainable params: 34196 (133.58 KB)
_________________________________________________________________


### For Finding Individual Feature of T-B/L-R

In [ ]:
# because rgb have 224X224x1 so we have to tweak model likewise

In [35]:
from keras.models import Model
intermediate_model = Model(inputs=densenet_model.input, outputs=densenet_model.get_layer('flatten').output)
intermediate_output_tl = intermediate_model.predict(tl_train)
# intermediate_output_tr = intermediate_model.predict(tr_train)
# intermediate_output_bl = intermediate_model.predict(bl)

28/28 [==============================] - 13s 428ms/step


In [36]:
intermediate_output_tl.shape

(877, 20480)

In [37]:
intermediate_output_tr = intermediate_model.predict(tr_train)

28/28 [==============================] - 11s 390ms/step


In [38]:
intermediate_output_tr.shape

(877, 20480)

In [39]:

intermediate_output_bl = intermediate_model.predict(bl_train)

28/28 [==============================] - 12s 429ms/step


In [40]:
intermediate_output_bl.shape

(877, 20480)

In [41]:
print(intermediate_model.input_shape)

(None, 128, 128, 3)


In [42]:
intermediate_output_br = intermediate_model.predict(br_train)

28/28 [==============================] - 12s 433ms/step


In [43]:
intermediate_output_br.shape

(877, 20480)

In [44]:
intermediate_output_tl_valid = intermediate_model.predict(tl_valid)

2/2 [==============================] - 0s 124ms/step


In [45]:
intermediate_output_tr_valid = intermediate_model.predict(tr_valid)

2/2 [==============================] - 1s 129ms/step


In [46]:
intermediate_output_bl_valid = intermediate_model.predict(bl_valid)

2/2 [==============================] - 0s 116ms/step


In [47]:
intermediate_output_br_valid = intermediate_model.predict(br_valid)

2/2 [==============================] - 0s 123ms/step


In [48]:
intermediate_output_tl_test = intermediate_model.predict(tl_test)

6/6 [==============================] - 3s 531ms/step


In [49]:
intermediate_output_tr_test = intermediate_model.predict(bl_test)

6/6 [==============================] - 2s 407ms/step


In [50]:
intermediate_output_bl_test = intermediate_model.predict(bl_test)

6/6 [==============================] - 3s 560ms/step


In [51]:
intermediate_output_br_test= intermediate_model.predict(br_test)

6/6 [==============================] - 2s 289ms/step


## Intermediate Model for Actual Image

In [52]:
densenet_model_main= Sequential()
densenet_model_main.add(base_model)
densenet_model_main.add(layers.Flatten(name='flatten'))
intermediate_features = densenet_model_main.layers[-1].output
attention_weights = layers.Dense(1, activation='softmax')(intermediate_features)
attention_weights = layers.Reshape((1, 1, -1))(attention_weights)
densenet_model_main.add(layers.Dense(10, activation='relu'))
densenet_model_main.layers[0].trainable = False

### For Finding Individual Feature of whole

In [ ]:
# because whole have 224X224x3 so we have to tweak model likewise

In [53]:
intermediate_model_2= Model(inputs=densenet_model_main.input, outputs=densenet_model_main.get_layer('flatten').output)


In [55]:
intermediate_output_main_train = intermediate_model_2.predict(X_train)

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueErro

In [ ]:
intermediate_output_main_valid = intermediate_model_2.predict(X_valid)

In [56]:
intermediate_output_main_test = intermediate_model_2.predict(X_test)

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueErro

In [ ]:
intermediate_output_main_train.shape

In [ ]:
intermediate_output_main_valid.shape

In [ ]:
intermediate_output_main_test.shape

## Combined Features for Fitting

In [ ]:
combined_features = []
for i in range(len(intermediate_output_bl)):
  combined_features.append(np.concatenate((intermediate_output_bl,intermediate_output_br,intermediate_output_tl, intermediate_output_tr), axis=0))
combined_features = np.array(combined_features)
combined_features.shape

In [ ]:
combined_features_valid = []
for i in range(len(intermediate_output_bl_valid)):
  combined_features_valid.append(np.concatenate((intermediate_output_bl_valid,intermediate_output_br_valid,intermediate_output_tl_valid, intermediate_output_tr_valid), axis=0))
combined_features_valid = np.array(combined_features_valid)
combined_features_valid.shape

In [ ]:
combined_features_test = []
for i in range(len(intermediate_output_bl_test)):
    combined_features_test.append(np.concatenate((intermediate_output_bl_test, intermediate_output_br_test, intermediate_output_tl_test, intermediate_output_tr_test), axis=0))
combined_features_test = np.array(combined_features_test)
combined_features_test.shape
